In [ ]:
!pip install kaggle
!pip install fastai==0.7.0
!pip install torchtext==0.2.3
!pip install pdpbox

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import pandas as pd
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import *
from sklearn.model_selection import *
from IPython.display import display

from sklearn.metrics import *
from scipy.cluster import hierarchy as hc
import seaborn as sns

from pdpbox import pdp
from plotnine import *
import feather

In [ ]:
%%time
train = pd.read_csv(f'../input/train_V2.csv')
test = pd.read_csv(f'../input/test_V2.csv')
samp_sub = pd.read_csv(f'../input/sample_submission_V2.csv')

In [ ]:
train.head()

In [ ]:
train.shape

# Build initial RF for data exploration

In [ ]:
#Take a smaller% sample of the data for training
perc = 0.1
train_small=train.sample(n = round(train.shape[0]*perc), random_state = 42)

In [ ]:
train.columns

In [ ]:
train_small.shape

In [ ]:
train_cats(train_small)
#Show that the categories have been successfully assigned
train_small.matchType.cat.categories

In [ ]:
df, y, nas = proc_df(train_small, 'winPlacePerc', max_n_cat=19)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#Create random training and testing samples
#Use only reduced columns of model to predict random forest 
X_train, X_test, y_train, y_test = train_test_split(
    df, y, 
    test_size=0.5, random_state=42)
print("X Train Shape: " + str(X_train.shape))
print("y Train Shape: " + str(y_train.shape))
print("X Test Shape: " + str(X_test.shape))
print("y Test Shape: " + str(y_test.shape))

In [ ]:
def print_score(m, X_train, y_train, X_test, y_test, verbose = True):
  if verbose:
    print("R**2 Score of model: " + str(m.score(X_train, y_train)))
    print("Mean Abs Error: " + str(mean_absolute_error(y_test, m.predict(X_test))))
    print("OOB Score: " + str(m.oob_score_))
  else:
    return m.score(X_train, y_train), mean_absolute_error(y_test, m.predict(X_test)), m.oob_score_;

In [ ]:
%%time 
#Sample run to test that random forrest is working and function above is working correctly

set_rf_samples(10000)
m = RandomForestRegressor(n_estimators = 40, 
                          random_state = 42,
                          criterion = 'mae', n_jobs = 4, 
                          min_samples_leaf=100, 
                          max_features = 'sqrt',
                          oob_score = True
                          )
m.fit(X_train, y_train)
print_score(m, X_train, y_train, X_test, y_test)

# Feature Engineering
### Analyze features in dataset and select/engineer important features

In [ ]:
#Check out variable importance of columns of training data
#Show the top 15 most impoartant variables from initial random forest
pd.DataFrame(m.feature_importances_, index = df.columns, columns=['importance']).sort_values(by = 'importance', ascending = False)[:15]

In [ ]:
fi = rf_feat_importance(m, df) #Print out top ten important features
fi[:30].plot('cols', 'imp', 'barh', figsize=(12,7), color='blue')

Generally the match type does not have any influence on the outcome of the players performance

In [ ]:
to_keep = fi[fi.imp>0.001].cols;
to_keep
#This gives the top 20 features

In [ ]:
#Use only reduced columns of model to predict random forest 
df_keep = train_small[to_keep].copy()
train_cats(df_keep)
df_keep, _, nas = proc_df(df_keep, max_n_cat=18)
print(df_keep.head())
X_train, X_test, y_train, y_test = train_test_split(
    df_keep, train_small.winPlacePerc, 
    test_size=0.5, random_state=42)
print("X Train Shape: " + str(X_train.shape))
print("y Train Shape: " + str(y_train.shape))
print("X Test Shape: " + str(X_test.shape))
print("y Test Shape: " + str(y_test.shape))

### Visualize some of the features and their relation to the dependent variable
A univariate plot is one way to show relation, but does not take into account other features interacting with the feature in question 

In [ ]:
df_keep_all=get_sample(df_keep.join(train_small.winPlacePerc), 500)
sns.regplot(x="walkDistance", y = 'winPlacePerc', data = df_keep_all, ci = 99)

In [ ]:
def plot_pdp(feat, clusters = None, feat_name = None):
  feat_name = feat_name or feat
  p = pdp.pdp_isolate(m, x, model_features = x.columns,feature=feat)
  return pdp.pdp_plot(p, feat_name, plot_lines = True,
                     cluster = clusters is not None, n_cluster_centers = clusters)

Re run the model with the reduced columns

In [ ]:
%%time 
set_rf_samples(10000)
m = RandomForestRegressor(n_estimators = 10, 
                          random_state = 42,
                          criterion = 'mae', n_jobs = 4, 
                          min_samples_leaf=100, 
                          max_features = 'sqrt')
m.fit(X_train, y_train)
print(m.score(X_train,y_train))
print(mean_absolute_error(y_test, m.predict(X_test)))
#learn here that removing some of the useless columns helps with model performance

In [ ]:
X_train.columns

Model improves score from 0.0931 to 0.861 by removing unecissary columns

In [ ]:
x=get_sample(X_train, 500)
plot_pdp(feat='maxPlace')

In [ ]:
feats = ['swimDistance', 'kills']
p = pdp.pdp_interact(m, x, model_features = x.columns, features = feats)
pdp.pdp_interact_plot(p, feats)

In [ ]:
%%time
corr = np.round(scipy.stats.spearmanr(get_sample(df_keep, 1000)).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method = 'average')
fig = plt.figure(figsize=(16,10))
dendogram = hc.dendrogram(z, labels = df_keep.columns, orientation='left', leaf_font_size = 16)
plt.show()

The dendogram plot shows that several of the variables are quite related:


*   kills - killStreaks
*   numGroups - maxPlace

Inicate we should investiate what these variables mean to see if we can eliminate any of them or consolidate them into more relevant features



In [ ]:
def get_oob(X_train, y_train):
    m = RandomForestRegressor(n_estimators = 10, 
                          random_state = 42,
                          criterion = 'mae', n_jobs = 4, 
                          min_samples_leaf=100, 
                          max_features = 'sqrt',
                             oob_score = True)
    m.fit(X_train, y_train)
    return m.oob_score_

In [ ]:
df_keep.columns

In [ ]:
#Recall that df_keep has a reduced number of columns that are relevant to selecting a winner
#Get a baseline
get_oob(X_train, y_train)

In [ ]:
for c in ('kills', 'killStreaks', 'numGroups', 'maxPlace'):
  print(c, get_oob(X_train.drop(c, axis = 1), y_train))

In [ ]:
to_drop = ['kills', 'numGroups']
get_oob(X_train.drop(to_drop, axis = 1), y_train)

## Feature selection (with all feature)

In [ ]:
%%time
m = RandomForestRegressor(n_estimators = 20, 
                          random_state = 42,
                          criterion = 'mae', n_jobs = 4, 
                          min_samples_leaf=100, 
                          max_features = 'sqrt',
                         oob_score = True)
m.fit(X_train, y_train)
print_score(m, X_train, y_train, X_test, y_test)
feats = ['walkDistance', 'killPlace', 'boosts', 'weaponsAcquired', 'heals',
       'longestKill', 'damageDealt', 'rideDistance', 'kills', 'assists',
       'swimDistance', 'headshotKills', 'matchDuration', 'numGroups', 'DBNOs',
       'maxPlace', 'killStreaks', 'revives', 'rankPoints', 'winPoints']
res = pd.DataFrame([],columns = ['Feature', 'R**2', 'MAE','OOB'])
r2, mae, oob = print_score(m, X_train, y_train, X_test, y_test, verbose = False)
res = res.append(pd.DataFrame({'Feature':['All Features'], 'R**2':[r2], "MAE":[mae], 'OOB':oob}))

In [ ]:
%%time
for f in feats:
    df_subs = X_train.drop(f, axis = 1)
    
    df_test = X_test.drop(f, axis = 1)
    m = RandomForestRegressor(n_estimators = 20, 
                          random_state = 42,
                          criterion = 'mae', n_jobs = 4, 
                          min_samples_leaf=100, 
                          max_features = 'sqrt', oob_score = True)
    m.fit(df_subs, y_train)
    r2, mae, oob = print_score(m, df_subs,y_train, df_test, y_test, verbose = False)
    res = res.append(pd.DataFrame({'Feature':[f], 'R**2':[r2], "MAE":[mae], 'OOB':oob}))

res.set_index('Feature')    
res.sort_values(by='MAE')

In [ ]:
def custom_style(row):
    font = 'normal'
    if row.values[1] == 'All Features':
        font = 'bold'
    return ['font-weight: %s' % font]*len(row.values)

res.reset_index().sort_values(by = "MAE").style.apply(custom_style, axis=1)

Run through all the top factors and see what removing some/many of them would do to the overall prediction of the model. It's shown that the factors of 'killStreaks', 'kills', and 'winPoints' when removed from the model increase the MAE score



In [ ]:
%%time
#Inital MAE score 0.0896
#drop_feats = ['killStreaks', 'swimDistance', 'assists','heals','rideDistance','DBNOs']
drop_feats = ['swimDistance']
df_subs = X_train.drop(drop_feats, axis = 1)  
df_test = X_test.drop(drop_feats, axis = 1)

m = RandomForestRegressor(n_estimators = 20, 
                          random_state = 42,
                          criterion = 'mae', n_jobs = 4, 
                          min_samples_leaf=100, 
                          max_features = 'sqrt',
                         oob_score = True)
m.fit(df_subs, y_train)
print_score(m, df_subs, y_train, df_test, y_test)

In [ ]:
fi = rf_feat_importance(m, df_subs) #Print out top ten important features
fi[:20].plot('cols', 'imp', 'barh', figsize=(12,7), color='blue')

Important to note here that killPlace has now supersceeded walkDistance as most important factor in predicting results

This improves our MAE score from 0.086 to 0.0826

In [ ]:
#Keep track of features that are going to be used for final model
kept_features = df_subs.columns
#kept_features = kept_features.append(pd.Index(['winPlacePerc']))

# Build Final Model for Submission

In [ ]:
train_f = train[kept_features]
train_cats(train_f)

In [ ]:
train_f.head()

In [ ]:
train_f.describe(include = 'all')

In [ ]:
df, _, nas = proc_df(train_f)

In [ ]:
print("training df shape: " +str(df.shape))
print("training y shape: " +str(train.winPlacePerc.shape))

In [ ]:
y = train.winPlacePerc.fillna(0).values

In [ ]:
from scipy import stats

In [ ]:
plt.hist(y)

In [ ]:
%%time
set_rf_samples(5000)
m = RandomForestRegressor(n_estimators = 100, 
                          random_state = 42,
                          criterion = 'mae', n_jobs = 4, 
                          min_samples_leaf=1, 
                          max_features = 'sqrt', oob_score = True)

m.fit(df, y)

In [ ]:
test_f = test[kept_features]
train_cats(test_f)

In [ ]:
test_df, _, nas = proc_df(test_f)

In [ ]:
test_df.head()

In [ ]:
pred_score = m.predict(test_df)
pred_score

In [ ]:
test.shape

In [ ]:
pred_score.shape

In [ ]:
samp_sub.head()

In [ ]:
my_submission = pd.DataFrame({'Id': test.Id, 'winPlacePerc': pred_score})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)